In [1]:
pip install pyarrow

     |████████████████████████████████| 17.3 MB 15.9 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install feather-format

  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2453 sha256=f835be9371e01bb3b2cc2594dc1d7072837400cb1695b59a5852014862a18373
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9d/90/64/88411837e80bb520d15ae395f39359a5413e7a00ae63397e10
Successfully built feather-format
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import json
import gzip
from urllib.request import urlopen
import feather 
import seaborn as sns

In [4]:
# function below prepares data by unzipping, dropping several columns that are full of nulls or unnecessary data for this
# particular analysis
def data_prep(data_name):
        
    file_name = f'{data_name}.json.gz'
    ### load the meta data
    data = []
    with gzip.open(file_name) as f:
        for l in f:
            data.append(json.loads(l.strip()))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)

    # drop columns containing nulls
    df = df.drop(columns=['reviewTime', 'asin', 'reviewerID', 'image', 'verified', 'style', 'summary', 'vote'])

    #print current shapes
    print(df.shape)
    
    feather_name = f'{data_name}.feather'

    # save in feather format
    df.to_feather(feather_name)

    #Convert unix timestamps to datetime.
    df['datetime'] = pd.to_datetime(df['unixReviewTime'],unit='s')
    
    # extract time-specific elements from datetime for further analysis    
    df['Weekday'] = df['datetime'].dt.dayofweek
    df['Month'] = df['datetime'].dt.month
    df['Day'] = df['datetime'].dt.day
    df['Year'] = df['datetime'].dt.year
    df['Quarter'] = df['datetime'].dt.quarter
    
    #drop the rows containing a null (should be a few hundred max)
    df = df.dropna()


    # shape should be 4-5+ more columsn depending on how many features you added above
    print(df.shape)
    
    return df

In [5]:
df_prime = data_prep('Prime_Pantry')

(471614, 4)
(471197, 10)


In [6]:
df_prime.head()

,overall,reviewerName,reviewText,unixReviewTime,datetime,Weekday,Month,Day,Year,Quarter
0,5.0,Tamara M.,Good clinging,1418515200,2014-12-14,6,12,14,2014,4
1,4.0,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,1416441600,2014-11-20,3,11,20,2014,4
2,4.0,noname,ok,1412985600,2014-10-11,5,10,11,2014,4
3,3.0,ZapNZs,Saran Cling Plus is kind of like most of the C...,1409529600,2014-09-01,0,9,1,2014,3
4,4.0,Amy Rogers,This is my go to plastic wrap so there isn't m...,1407628800,2014-08-10,6,8,10,2014,3


In [7]:
df_prime['reviewText'][4]

"This is my go to plastic wrap so there isn't much bad I can say. Plastic wrap is kind of a pain when you are trying to get it off the roll and it sticks to itself. I don't seem to have that problem with cling plus. It comes out nice and rips off well. It usually sticks well to what i need it to. I will say i use it more to wrap things, then to go over containers. I just don't feel any plastic wrap sticks really well to plastic and glass bowls. So i use it sparingly for these purposes. If i need to wrap up something for my children's lunch this is the plastic wrap i would choose."

In [8]:
review_length = []

for line in df_prime['reviewText']:
    review_length.append(len(line))

In [9]:
df_prime['Review Length'] = review_length

In [10]:
df_prime['Review Length'].describe()

count    471197.000000
mean        133.276776
std         211.008861
min           1.000000
25%          25.000000
50%          66.000000
75%         155.000000
max       10176.000000
Name: Review Length, dtype: float64

In [12]:
df_prime.loc[df_prime['Review Length'] == 1]

,overall,reviewerName,reviewText,unixReviewTime,datetime,Weekday,Month,Day,Year,Quarter,Review Length
273,5.0,Louie,k,1439424000,2015-08-13,3,8,13,2015,3,1
342,5.0,Amazon Customer,1,1428710400,2015-04-11,5,4,11,2015,2,1
5768,4.0,Amazon Customer,a,1522108800,2018-03-27,1,3,27,2018,1,1
5836,5.0,JAMES,a,1504656000,2017-09-06,2,9,6,2017,3,1
6435,5.0,Lucy&#039;s mom!!,A,1421193600,2015-01-14,2,1,14,2015,1,1
...,...,...,...,...,...,...,...,...,...,...,...
466565,4.0,joy trammell,k,1528329600,2018-06-07,3,6,7,2018,2,1
466999,5.0,Amazon Customer,k,1504310400,2017-09-02,5,9,2,2017,3,1
467336,4.0,joy trammell,k,1528329600,2018-06-07,3,6,7,2018,2,1
469282,5.0,Texas_Aggie,A,1508716800,2017-10-23,0,10,23,2017,4,1


In [16]:
df_prime[df_prime['Review Length'] == 1]['reviewText'].unique()

array(['k', '1', 'a', 'A', 'C', 'F', '4', '5', 'd', '.', 'f', 'c', 'K',
       'B', 'b', ' ', '7', 'e'], dtype=object)

At first glance, the `reviewText` column of just single characters seemed like an odd, even random collection, especially when looking at a nonrepeating unique list of which characters were used. But when we do a tally below of how often each character was used, it suddenly makes a lot more sense. The sum of the first 4 characters - `'a', 'A', '1', 'k'` - account for over 80% of the entries. This tells me that people were being asked to input a review and they just wanted to skip that step so they stuck something there. 

In [24]:
print((90 + 87 + 83 + 44) / 368)
print(df_prime[df_prime['Review Length'] == 1]['reviewText'].count())

df_prime[df_prime['Review Length'] == 1]['reviewText'].value_counts()

0.8260869565217391
368


a    90
A    87
1    83
k    44
d    17
f     8
K     6
B     6
.     6
4     5
C     4
5     4
c     2
F     2
      1
e     1
7     1
b     1
Name: reviewText, dtype: int64

What I'm curious to see is how many A) occurances and B) variants of `n/a` there are. 

Looking for `'na' - 'n/a' - 'NA' - 'N/A'` here

In [25]:
df_prime[df_prime['Review Length'] == 2]['reviewText'].count()

2401

In [26]:
df_prime[df_prime['Review Length'] == 2]['reviewText'].unique()

array(['ok', 'ty', 'a+', 'A+', 'Ok', 'a1', 'OK', ':)', 'Ty', '.k', 'no',
       'TY', '<3', ':0', 'NO', 'C!', 'aa', '=)', '#1', 'ew', '(:', 'gd',
       'No', 'SO', 'A=', 'ya', 'so', 'Hi', ';)', 'A1', 'VG', '5*', 'vg',
       'AA', 'hi', 'tx', 'A!', ':P', 'NC', '^^', 'eh', 'Me', ';0', '!!',
       '#2', ':}', 'ha', 'TP', 'Eh', ':D', '..', 'Ew'], dtype=object)

In [27]:
df_prime[df_prime['Review Length'] == 3]['reviewText'].count()

1570

In [28]:
df_prime[df_prime['Review Length'] == 3]['reviewText'].unique()

array(['Ty!', 'AAA', 'yum', 'ok.', 'Yum', 'Duh', 'God', 'Eww', 'A++',
       'YUM', 'yes', 'Meh', ':-)', 'Ok.', ':-D', 'yuk', 'yup', 'Yes',
       '***', 'Fab', 'THX', ';-)', 'bye', 'Fav', 'A-1', 'old', 'Lov',
       'PBJ', 'Umm', 'aok', 'yay', 'yep', 'thx', 'Yuk', 'wow', 'meh',
       'joy', 'Fun', 'BOB', 'nom', 'Gr8', ':D ', 'gr8', 'Yup', 'GTG',
       'YES', 'ugg', 'new', 'god', 'Wow', 'Tks', 'umm', 'I  ', 'Num',
       'Thx', 'woo', 'OK.', 'omg', 'FYI', 'ook', ' it', '( :', 'OK,',
       'A#1', 'Aok', 'YUP', 'bad', 'NTS', 'aaa', 'AOK', 'H2o', 'H20',
       'H2O', 'YEP', 'gtg', 'YAY', 'Buy', 'big', 'dry', 'Dry', 'WOW',
       'ATE', 'gas', 'OMG', 'sty', 'YUK', 'fgk', 'Yep', '5/5', 'out',
       'Ok!', 'ok!', ': )', 'hot', 'fab', 'MSG', 'tea', 'a +', 'Met',
       'ugh', 'ok-', 'fun', 'o.k', 'mmk', '1!!', 'Use', 'WIN', 'A +',
       'fav', '" )', ':))', '= )', ':0)', 'nah', 'Grt', 'Eh!', 'Old',
       'YEA', 'Mmm', 'Woe', 'eh!', 'Pav', 'luv', 'Sof', 'Bad', 'lov',
       'YEs', 'oko',

In [35]:
df_prime['Review Length'].value_counts()[0:15]

11    8352
10    8124
4     7806
12    7616
5     7545
9     7482
13    6431
14    5048
17    4782
15    4657
18    4645
16    4381
19    4347
20    4206
25    4050
Name: Review Length, dtype: int64

In [37]:
df_prime[df_prime['Review Length'] == 11]['reviewText'].value_counts()[0:15]

As expected    450
as expected    439
Great price    295
works great    270
great price    249
Works great    239
Great taste    188
Great value    154
great taste    137
Good stuff.    113
Taste great    103
taste great     97
Love these!     92
Works well.     91
great value     90
Name: reviewText, dtype: int64

In [38]:
df_prime[df_prime['Review Length'] == 10]['reviewText'].value_counts()[0:15]

Delicious!    604
good stuff    315
good price    270
Good stuff    251
Good price    193
Very good.    185
Excellent!    176
Delicious.    167
works well    153
Works well    151
Very good!    137
Love these    130
Good value    114
Thank you!    113
good value    111
Name: reviewText, dtype: int64

In [39]:
df_prime['reviewText'].value_counts()[0:15]

good             2789
Good             2122
great            1964
Great            1726
ok               1142
Great product     804
love it           673
Great!            660
Delicious!        604
Love it           580
Delicious         560
yummy             554
Yummy             541
very good         538
Good product      535
Name: reviewText, dtype: int64

In [40]:
df_prime[df_prime['Review Length'] == 4]['reviewText'].value_counts()[0:15]

good    2789
Good    2122
Yum!     309
nice     303
love     266
Nice     232
GOOD     152
Love     127
fine     120
okay      80
Yum.      61
Okay      60
YUM!      57
none      54
Fine      52
Name: reviewText, dtype: int64

I'm curious how similar these metrics are across a couple other catagories like instruments and fashion

### Instruments exploration

In [41]:
df_instruments = data_prep('Musical_Instruments')

(1512530, 4)
(1511526, 10)


In [46]:
review_length = []

for line in df_instruments['reviewText']:
    review_length.append(len(line))
    
df_instruments['Review Length'] = review_length

In [48]:
print(df_instruments[df_instruments['Review Length'] == 1]['reviewText'].count())
print(df_instruments[df_instruments['Review Length'] == 1]['reviewText'].unique())
df_instruments[df_instruments['Review Length'] == 1]['reviewText'].value_counts()

316
['d' 'k' 'A' '1' 'a' 'c' 'E' 'K' 'C' 'D' 'e' 'B' 'f' 'b' ' ' '5' '.' '^'
 'g']


a    77
A    75
k    46
1    42
K    23
d    14
f     8
c     8
B     3
D     3
C     3
e     3
      2
.     2
5     2
E     2
g     1
^     1
b     1
Name: reviewText, dtype: int64

In [49]:
(77 + 75 + 46 + 42) / 316

0.759493670886076

In [51]:
df_instruments[df_instruments['Review Length'] == 2]['reviewText'].unique()

array(['ok', ':)', 'Ok', 'a1', 'A+', 'OK', 'ty', 'a+', 'Go', 'AA', 'C+',
       'Mm', 'No', '#1', 'Yo', '<3', 'ej', 'aa', 'A=', '%0', 'no', 'B-',
       '5*', 'eh', 'Fr', 'vg', '=D', 'Aa', ';)', '4*', 'A1', '1*', 'VG',
       'Eh', ':D', 'Hi', 'TY', 'It', 'ew', 'Su', 'Ty', '=(', '=)', 'NO',
       'F-', 'D-', '(:', 'Vv', '+1', 'VE', 'Me', ':(', ':/', 'nc', 'ye',
       ';(', 'np', '!!', 'Ye', 'gg', ':>', 'ic', 'Gg', 'So', 'Ju'],
      dtype=object)

In [52]:
df_instruments['Review Length'].value_counts()[0:15]

12    13954
9     13558
11    12769
10    12224
13    11218
4     10293
7      9267
18     9135
5      8714
28     8611
14     8599
27     8506
17     8294
29     8294
19     8285
Name: Review Length, dtype: int64

In [55]:
df_instruments['reviewText'].value_counts()[0:15]

good             2923
Great            2896
Good             2596
great            2359
Excellent        1867
Works great      1567
Perfect          1557
Great product    1328
Great!           1315
ok               1292
works great      1194
Love it          1177
Works great!     1165
Nice             1111
nice             1016
Name: reviewText, dtype: int64

### Fashion exploration

In [42]:
df_fashion = data_prep('AMAZON_FASHION')

(883636, 4)
(882313, 10)


In [56]:
review_length = []

for line in df_fashion['reviewText']:
    review_length.append(len(line))
    
df_fashion['Review Length'] = review_length

In [57]:
print(df_fashion[df_fashion['Review Length'] == 1]['reviewText'].count())
print(df_fashion[df_fashion['Review Length'] == 1]['reviewText'].unique())
df_fashion[df_fashion['Review Length'] == 1]['reviewText'].value_counts()

171
['A' 'D' 'a' 'k' 'c' '1' 'E' 'd' 'C' 'e' 'K' '.' 'F' 'b' 'B' ' ' 'f' 'm'
 'N']


A    44
a    29
k    25
1    23
K    12
c    10
C     5
d     5
.     4
E     3
b     2
B     2
      1
e     1
f     1
D     1
F     1
m     1
N     1
Name: reviewText, dtype: int64

In [61]:
(44 + 29 + 25 + 23) / 171

0.7076023391812866

In [58]:
df_fashion[df_fashion['Review Length'] == 2]['reviewText'].unique()

array(['ok', 'Ok', ':)', 'A+', 'OK', 'No', 'A1', ':(', ';)', '<3', 'ty',
       'F-', 'no', '(:', '0k', '0+', 'a+', 'bb', 'TY', 'Nc', '4*', 'Hi',
       ':D', 'Ty', 'AA', ':]', ':/', 'Xx', ':P', 'NO', '=]', '1*', 'Eh',
       'hi', 'A-', '+1', 'A*', 'I.', '5*', ':i', 'eh', '++', ':*', '\n\n',
       'Ew', 'vg', '=)', 'It', 'tu', '3*', 'a*', 'Sq', 'OE', 'Wa', 'NA',
       'be', '-+', '+2', '=(', 'A=', '11', 'Mm', ';('], dtype=object)

In [59]:
df_fashion['Review Length'].value_counts()[0:15]

9     14026
10     9830
7      9034
4      8319
11     7899
8      7802
12     7719
13     6923
14     6608
27     6454
17     6433
28     6327
26     6292
19     6253
25     6232
Name: Review Length, dtype: int64

In [60]:
df_fashion['reviewText'].value_counts()[0:15]

Love it      2658
Great        1650
Good         1517
Nice         1516
Love it!     1405
Perfect      1267
good         1136
love it      1117
nice          879
great         875
Beautiful     841
Too small     748
Very nice     738
Excellent     737
I love it     674
Name: reviewText, dtype: int64

### Exploration of top 15 most common reviews across several catagories 

Code below

| Prime         | Instruments   | Fashion   | Beauty         | Appliances    | Software      |
|---------------|---------------|-----------|----------------|---------------|---------------|
| good          | good          | Love it   | Good           | Works great   | good          |
| Good          | Great         | Great     | Love it        | Great         | Great         |
| great         | Good          | Good      | Great          | Perfect       | great         |
| Great         | great         | Nice      | good           | Good          | Good          |
| ok            | Excellent     | Love it!  | Great product  | Perfect fit   | ok            |
| Great product | Works great   | Perfect   | great          | good          | Excellent     |
| love it       | Perfect       | good      | Love it!       | works great   | Great product |
| Great!        | Great product | love it   | Excellent      | great         | Love it       |
| Delicious!    | Great!        | nice      | love it        | Works great!  | love it       |
| Love it       | ok            | great     | Nice           | Excellent     | Works great   |
| Delicious     | works great   | Beautiful | Great product! | Great product | works great   |
| yummy         | Love it       | Too small | Works great    | Worked great  | Great!        |
| Yummy         | Works great!  | Very nice | Perfect        | perfect       | excellent     |
| very good     | Nice          | Excellent | Good product   | Works great.  | great product |
| Good product  | nice          | I love it | ok             | perfect fit   | Good product  |


Couple interesting oberservations from this:
- the top 4 are basically the same across catagories
- the further down the list you go, the most catagory specific a review becomes. A few that stick out are:
    - "Yum!" and "yummy" in Prime
    - "works great" and "great product" in Instruments, Appliances, Software, and Beauty. I have to go down to the 30+ range to find similar reviews in Fashion

In [62]:
df_prime['reviewText'].value_counts()[0:15]

good             2789
Good             2122
great            1964
Great            1726
ok               1142
Great product     804
love it           673
Great!            660
Delicious!        604
Love it           580
Delicious         560
yummy             554
Yummy             541
very good         538
Good product      535
Name: reviewText, dtype: int64

In [63]:
df_instruments['reviewText'].value_counts()[0:15]

good             2923
Great            2896
Good             2596
great            2359
Excellent        1867
Works great      1567
Perfect          1557
Great product    1328
Great!           1315
ok               1292
works great      1194
Love it          1177
Works great!     1165
Nice             1111
nice             1016
Name: reviewText, dtype: int64

In [70]:
df_fashion['reviewText'].value_counts()[46:60]

Perfect fit       211
Very pretty       207
love them         198
Love it!!!        193
Super cute!       193
Thank you         190
To small          184
Great product!    184
Cute!             182
Very nice.        182
Poor quality      177
love it!          176
Very nice!        175
Fits great        175
Name: reviewText, dtype: int64

In [65]:
df_beauty = data_prep('All_Beauty')
df_beauty['reviewText'].value_counts()[0:15]

(371345, 4)
(370908, 10)


Good              802
Love it           787
Great             776
good              756
Great product     564
great             474
Love it!          459
Excellent         389
love it           386
Nice              347
Great product!    305
Works great       287
Perfect           286
Good product      275
ok                269
Name: reviewText, dtype: int64

In [66]:
df_appliances = data_prep('Appliances')
df_appliances['reviewText'].value_counts()[0:15]

(602777, 4)
(602438, 10)


Works great      1575
Great            1393
Perfect          1354
Good             1258
Perfect fit      1153
good             1134
works great      1016
great             987
Works great!      973
Excellent         758
Great product     754
Worked great      660
perfect           653
Works great.      618
perfect fit       612
Name: reviewText, dtype: int64

In [67]:
df_software = data_prep('Software')
df_software['reviewText'].value_counts()[0:15]

(459436, 4)
(459346, 10)


good             811
Great            775
great            728
Good             624
ok               575
Excellent        462
Great product    313
Love it          294
love it          275
Works great      264
works great      261
Great!           253
excellent        214
great product    208
Good product     193
Name: reviewText, dtype: int64